In [1]:
#!/usr/bin/env python3

import os
import django
import json
import sys
import requests

# Set up Django environment
sys.path.append('/home/anveg/Development/Tech_assessment/reimagined')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mindtimer.settings')
django.setup()

from dotenv import load_dotenv
from tasks.services import ClaudeTaskGroomer

load_dotenv()

True

In [2]:
# Check API key
api_key = os.getenv('CLAUDE_API_KEY')
if not api_key:
    print("❌ CLAUDE_API_KEY not found in environment!")

print(f"✅ API Key found: {api_key[:15]}...")
print(f"🔍 API Key format check: {'✅ Correct (starts with sk-ant-)' if api_key.startswith('sk-ant-') else '❌ Invalid format'}")


✅ API Key found: sk-ant-api03-gf...
🔍 API Key format check: ✅ Correct (starts with sk-ant-)


In [3]:
# Test different endpoints to diagnose 404 issue
base_url = "https://api.anthropic.com"
endpoints_to_test = [
    "/v1/messages",
    "/v1/complete", 
    "/health",
    "/"
]

headers = {
    'Content-Type': 'application/json',
    'x-api-key': api_key,
    'anthropic-version': '2023-06-01'
}

print("\n🔍 Testing different endpoints...")
for endpoint in endpoints_to_test:
    url = base_url + endpoint
    #try:
    # Try a simple GET first to see what endpoints exist
    response = requests.get(url, headers=headers, timeout=10)
    print(f"GET {endpoint}: {response.status_code} - {response.reason}")
    
    if endpoint == "/v1/messages":
        # Try the actual POST request
        payload = {
            "model": "claude-3-haiku-20240307",
            "max_tokens": 1024,
            "messages": [{"role": "user", "content": "Hello"}]
        }
        post_response = requests.post(url, headers=headers, json=payload, timeout=10)
        print(f"POST {endpoint}: {post_response.status_code} - {post_response.reason}")
        
        if post_response.status_code != 200:
            try:
                error_details = post_response.json()
                print(f"    Error details: {json.dumps(error_details, indent=2)}")
            except:
                print(f"    Response text: {post_response.text[:200]}...")
        else:
            print("    ✅ Messages API working!")


🔍 Testing different endpoints...
GET /v1/messages: 405 - Method Not Allowed
POST /v1/messages: 200 - OK
    ✅ Messages API working!
GET /v1/complete: 405 - Method Not Allowed
GET /health: 404 - Not Found
GET /: 404 - Not Found


In [4]:
# Test TaskGroomer with actual groom_tasks function
print("\n🧪 Testing TaskGroomer.groom_tasks()...")
groomer = ClaudeTaskGroomer()
test_todo = "Do laundry\nBuy groceries"
test_context = "test context"

print(f"🔍 Calling groomer.groom_tasks('{test_todo}', '{test_context}')")
result = groomer.groom_tasks(test_todo, test_context)


🧪 Testing TaskGroomer.groom_tasks()...
🔍 Calling groomer.groom_tasks('Do laundry
Buy groceries', 'test context')
direct result from Claude: {'id': 'msg_01V58ycyHVwzJs4yuJbd9HqD', 'type': 'message', 'role': 'assistant', 'model': 'claude-sonnet-4-20250514', 'content': [{'type': 'text', 'text': '```json\n{\n  "tasks": [\n    {\n      "task": "Sort dirty clothes into appropriate loads (whites, colors, delicates)",\n      "gen_task_id": "L001",\n      "time_estimate": "00:15",\n      "dependencies": [],\n      "priority": "medium"\n    },\n    {\n      "task": "Load first batch of laundry into washing machine and start cycle",\n      "gen_task_id": "L002",\n      "time_estimate": "00:05",\n      "dependencies": ["L001"],\n      "priority": "medium"\n    },\n    {\n      "task": "Transfer first load from washer to dryer and start drying cycle",\n      "gen_task_id": "L003",\n      "time_estimate": "00:05",\n      "dependencies": ["L002"],\n      "priority": "medium"\n    },\n    {\n      "t

In [5]:
# Test TaskGroomer with actual groom_tasks function
print("\n🧪 Testing TaskGroomer.groom_tasks()...")
groomer = ClaudeTaskGroomer()
test_todo = "I am overwhelmed with school work. I have to study for my exams next week, I have a group project and my team mates are disorganized. I also have to figure out what I am going to make for dinner and I cant focus. "
test_context = "I have a headache, its too much I dont even know where to start. "

print(f"🔍 Calling groomer.groom_tasks('{test_todo}', '{test_context}')")
result = groomer.groom_tasks(test_todo, test_context)


🧪 Testing TaskGroomer.groom_tasks()...
🔍 Calling groomer.groom_tasks('I am overwhelmed with school work. I have to study for my exams next week, I have a group project and my team mates are disorganized. I also have to figure out what I am going to make for dinner and I cant focus. ', 'I have a headache, its too much I dont even know where to start. ')
direct result from Claude: {'id': 'msg_01Cuzq8THuxYb93TaG1QhQwd', 'type': 'message', 'role': 'assistant', 'model': 'claude-sonnet-4-20250514', 'content': [{'type': 'text', 'text': '```json\n{\n  "tasks": [\n    {\n      "task": "Take 15 minutes to rest and address headache with water, pain relief, or brief break",\n      "gen_task_id": "REST_001",\n      "time_estimate": "00:15",\n      "dependencies": [],\n      "priority": "high"\n    },\n    {\n      "task": "List all upcoming exams with dates and subjects",\n      "gen_task_id": "EXAM_001",\n      "time_estimate": "00:10",\n      "dependencies": ["REST_001"],\n      "priority": "hig